In [2]:
import pandas as pd
from utils.notebookhelpers.helpers import Helpers
from utils.dtos.templateOutputCollection import TemplateOutputCollection
from utils.dtos.variable import Metadata
from utils.dtos.templateOutput import TemplateOutput
from utils.dtos.templateOutput import OutputType
from utils.dtos.templateOutput import ChartType
import pandas as pd
import pickle
import numpy as np
!pip install scikit-learn
# import plotly.express as px
from sklearn.model_selection import GridSearchCV,KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import  f1_score, roc_auc_score, confusion_matrix
from shaphypetune import BoostSearch, BoostBoruta, BoostRFE, BoostRFA
from sklearn.metrics import roc_auc_score
from scipy import stats
# from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

##Pycaret
# import jinja2
# from pycaret.classification import *
import pickle
import copy
import numpy as np

In [21]:
#Getting the context
contextId = 'Airline_performance'
context = Helpers.getOrCreateContext(contextId=contextId, localVars=locals())

inputDatasetParameter=Helpers.get_or_create_input_dataset(
    name="inputDataset3",
    metadata=Metadata(input_name='Clustered data', is_required=True, tooltip='output transformation'),
    local_context=locals()
)






Departures_On_Time_Parameter = Helpers.get_or_create_input_var(
    name="Departures_On_Time",
    metadata=Metadata(input_name="Departures_On_Time", is_required=True, tooltip="All Departures_On_Time", multiple=False,datatypes=['STRING'] ,options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)




Total_scheduled_Parameter = Helpers.get_or_create_input_var(
    name="Total_scheduled",
    metadata=Metadata(input_name="Total_scheduled", is_required=True, tooltip="All Departures_On_Time", multiple=False, datatypes=['STRING'],options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)



Total_departed_Parameter = Helpers.get_or_create_input_var(
    name="Total_departed",
    metadata=Metadata(input_name="Total_departed", is_required=True, tooltip="All Departures_On_Time", multiple=False, datatypes=['STRING'],options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)





Cancellations_Parameter = Helpers.get_or_create_input_var(
    name="Cancellations",
    metadata=Metadata(input_name="Cancellations", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)


Arrivals_Delayed_Parameter = Helpers.get_or_create_input_var(
    name="Arrivals_Delayed",
    metadata=Metadata(input_name="Arrivals_Delayed", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)





Cancelled_perc_Parameter = Helpers.get_or_create_input_var(
    name="Cancelled_perc",
    metadata=Metadata(input_name="Cancelled_perc", is_required=True, tooltip="Output column", multiple=False,
),
    local_context=locals()
)


departure_delay_perc_Parameter = Helpers.get_or_create_input_var(
    name="departure_delay_perc",
    metadata=Metadata(input_name="departure_delay_perc", is_required=True, tooltip="Output column", multiple=False, 
),
    local_context=locals()
)


arrival_delay_perc_Parameter = Helpers.get_or_create_input_var(
    name="arrival_delay_perc",
    metadata=Metadata(input_name="arrival_delay_perc", is_required=True, tooltip="Output column", multiple=False,
),
    local_context=locals()
)


Dep_Arr_delay_Parameter = Helpers.get_or_create_input_var(
    name="Dep_Arr_delay",
    metadata=Metadata(input_name="Dep_Arr_delay", is_required=True, tooltip="Output column", multiple=False,
),
    local_context=locals()
)



Btwness_Parameter = Helpers.get_or_create_input_var(
    name="Btwness",
    metadata=Metadata(input_name="Btwness", is_required=True, tooltip="Output column", multiple=False,
),
    local_context=locals()
)


closeness_Parameter = Helpers.get_or_create_input_var(
    name="closeness",
    metadata=Metadata(input_name="closeness", is_required=True, tooltip="Output column", multiple=False, 
),
    local_context=locals()
) 


Deg_centrality_Parameter = Helpers.get_or_create_input_var(
    name="Deg_centrality",
    metadata=Metadata(input_name="Deg_centrality", is_required=True, tooltip="Output column", multiple=False, 
),
    local_context=locals()
) 






outputDatasetParameter=Helpers.get_or_create_output_dataset(
 name="outputDataset3",
    metadata=Metadata(input_name='Xtrain', is_required=True, tooltip='Dataset name to be created after the transformation'),
    local_context=locals()
)


#Model Save
modelNameParameter=Helpers.get_or_create_input_var(
    name="modelName",
    metadata=Metadata(input_name="Published Model Name", is_required=True, tooltip="Type the model name that you have saved", multiple=False, datatypes=['STRING'], options=['CONSTANT']),
    local_context=locals()
)

In [22]:
#Getting the parameters
airline_val=inputDatasetParameter.value
Departures_On_Time =Departures_On_Time_Parameter.value

Cancellations=    Cancellations_Parameter.value

Arrivals_Delayed = Arrivals_Delayed_Parameter.value

Total_scheduled=Total_scheduled_Parameter.value
Total_departed= Total_departed_Parameter.value
Cancelled_perc = Cancelled_perc_Parameter.value
departure_delay_perc= departure_delay_perc_Parameter.value
arrival_delay_perc = arrival_delay_perc_Parameter.value
Dep_Arr_delay = Dep_Arr_delay_Parameter.value
Btwness = Btwness_Parameter.value
closeness = closeness_Parameter.value
Deg_centrality = Deg_centrality_Parameter.value

outputDataset3=outputDatasetParameter.value

modelName=modelNameParameter.value

In [23]:
df = Helpers.getEntityData(context,airline_val)


In [33]:
df.head(2)

,Deg_centrality,closeness,Btwness,Cancelled_perc,departure_delay_perc,arrival_delay_perc,Total_scheduled,Total_departed,Cancellations,Departures_On_Time,Arrivals_Delayed,Dep_Arr_delay
0,0.270934,0.448209,0.209187,0.266232,2.697741,2.611200,8.410943,8.403352,3.555348,8.254789,6.278521,2.890372
1,0.217674,0.397667,0.140924,0.203556,2.322388,2.424803,10.404929,10.399494,5.192957,10.295327,8.155362,0.000000


In [25]:
sc = StandardScaler()
val = sc.fit_transform(df)
# X_test_norm = sc.transform(X_test)

In [26]:
val[:5]

array([[ 0.5655881 ,  0.45877765,  0.72548992, -0.69100003, -0.21591725,
        -0.47192671,  1.32063335,  1.33007551,  0.81916939,  1.35761786,
         1.06381609,  1.53308623],
       [-0.0256263 , -0.26902114, -0.11234399, -0.78121294, -0.93380305,
        -0.83172873,  2.58482288,  2.5942067 ,  1.80182731,  2.64540784,
         2.21708883, -0.72241099],
       [-0.50247196, -0.23623651, -0.18817239,  0.8493029 , -0.72107722,
        -0.95516849, -0.02422772, -0.02392525,  0.26934311,  0.02560944,
        -0.28863477,  0.35938171],
       [ 0.75149775,  0.58507816,  0.55075729, -0.17120876, -0.69515634,
        -1.177977  ,  0.96159163,  0.97126275,  0.51264512,  1.02235916,
         0.56059999,  1.61530415],
       [ 0.07537989,  0.15676898, -0.27442133,  0.06983194, -1.21616594,
        -1.4983769 ,  0.36824391,  0.37784611,  0.06744073,  0.443806  ,
        -0.1015336 ,  0.53351144]])

In [30]:
 Helpers.downloadArtifacts(context, modelName)

_request non-retriable exception: Anonymous caller does not have storage.objects.list access to the Google Cloud Storage bucket. Permission 'storage.objects.list' denied on resource (or it may not exist)., 401
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/gcsfs/retry.py", line 115, in retry_request
    return await func(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/gcsfs/core.py", line 384, in _request
    validate_response(status, contents, path, args)
  File "/opt/conda/lib/python3.10/site-packages/gcsfs/retry.py", line 102, in validate_response
    raise HttpError(error)
gcsfs.retry.HttpError: Anonymous caller does not have storage.objects.list access to the Google Cloud Storage bucket. Permission 'storage.objects.list' denied on resource (or it may not exist)., 401


downloading artifacts from gs://rapidcanvas-platform-data-local/tenant-test-tenant/artifacts/XGB_Shap_model/ to /tmp/rc/temp/tenant-test-tenant/artifacts/XGB_Shap_model/


{}

In [27]:
filesDict = Helpers.downloadArtifacts(context, modelName)
XGB_clf = pickle.load(open(filesDict["XGB_Shap_.pkl"], 'rb'))

_request non-retriable exception: Anonymous caller does not have storage.objects.list access to the Google Cloud Storage bucket. Permission 'storage.objects.list' denied on resource (or it may not exist)., 401
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/gcsfs/retry.py", line 115, in retry_request
    return await func(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/gcsfs/core.py", line 384, in _request
    validate_response(status, contents, path, args)
  File "/opt/conda/lib/python3.10/site-packages/gcsfs/retry.py", line 102, in validate_response
    raise HttpError(error)
gcsfs.retry.HttpError: Anonymous caller does not have storage.objects.list access to the Google Cloud Storage bucket. Permission 'storage.objects.list' denied on resource (or it may not exist)., 401


downloading artifacts from gs://rapidcanvas-platform-data-local/tenant-test-tenant/artifacts/XGB_Shap_model/ to /tmp/rc/temp/tenant-test-tenant/artifacts/XGB_Shap_model/


KeyError: 'XGB_Shap_.pkl'

In [31]:
len(df)

1587

In [ ]:
df.head()

In [32]:
df2 = np.expm1(df)
df2['predicted'] = XGB_clf.predict(val)

NameError: name 'XGB_clf' is not defined

In [ ]:
outputCollection = Helpers.createOutputCollection(context)
out = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset3, dataFrame=df2)

# out = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset3, dataFrame=Xtrain)
# out2 = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset4, dataFrame=X_test_norm)
# out3 = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset5, dataFrame=Y_train)
# out4 = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset6, dataFrame=Y_test)
outputCollection.addTemplateOutput(out)
# outputCollection.addTemplateOutput(out2)
# outputCollection.addTemplateOutput(out3)
# outputCollection.addTemplateOutput(out4)
Helpers.save(context)